In [23]:
# To Do
# Succesfully open and read one of the data files
# Inspect the columns and their data types
# Write code that creates the table for the data
# Upload the data to the table

In [24]:
# Importing Dependencies
import pandas as pd
import os

In [41]:


# Cycle through the files in Resources and append 100 rows from each
dirLoc = 'Resources' #filepath of the directory
df = pd.DataFrame()

for file in os.scandir(dirLoc):

    if "Morti" in file.path: # Only append files from the Mortality Datasets
        df = df.append(pd.read_csv(file.path, nrows = 100))
        
        

In [42]:
df

,Country,Admin1,SubDiv,Year,List,Cause,Sex,Frmat,IM_Frmat,Deaths1,...,Deaths21,Deaths22,Deaths23,Deaths24,Deaths25,Deaths26,IM_Deaths1,IM_Deaths2,IM_Deaths3,IM_Deaths4
0,1400,NaN,NaN,2001,101,1000,1,7,8,332,...,95,NaN,NaN,NaN,NaN,0,8,NaN,NaN,NaN
1,1400,NaN,NaN,2001,101,1000,2,7,8,222,...,112,NaN,NaN,NaN,NaN,0,11,NaN,NaN,NaN
2,1400,NaN,NaN,2001,101,1001,1,7,8,24,...,5,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN
3,1400,NaN,NaN,2001,101,1001,2,7,8,14,...,6,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN
4,1400,NaN,NaN,2001,101,1002,1,7,8,0,...,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4303,NaN,NaN,2017,101,1049,2,1,8,66,...,4,1.0,2.0,NaN,NaN,0,0,NaN,NaN,NaN
96,4303,NaN,NaN,2017,101,1050,1,1,8,77,...,6,1.0,0.0,NaN,NaN,0,10,NaN,NaN,NaN
97,4303,NaN,NaN,2017,101,1050,2,1,8,49,...,3,2.0,1.0,NaN,NaN,0,7,NaN,NaN,NaN
98,4303,NaN,NaN,2017,101,1051,1,1,8,966,...,91,36.0,14.0,NaN,NaN,0,10,NaN,NaN,NaN
